# LSTMを用いた株価予測デモ

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from datetime import datetime, timedelta
from pandas_datareader import data as pdr

In [2]:
# Define the stock symbol and timeframe
symbol = '6920.T' 
end_date = datetime.today()
start_date = end_date - timedelta(days=365)
yf.pdr_override()
# yahooサイトからデータをダウンロード
data = pdr.get_data_yahoo(symbol, start_date, end_date)
data.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-12-27,22890.0,23195.0,22605.0,22980.0,22792.529297,10622800
2022-12-28,22480.0,22525.0,21880.0,22290.0,22108.158203,10026100
2022-12-29,21740.0,22560.0,21725.0,22515.0,22383.541016,10521200
2022-12-30,22905.0,23080.0,21700.0,21745.0,21618.037109,12068100
2023-01-04,21395.0,21650.0,20975.0,21150.0,21026.509766,8868500


In [3]:
close_prices = data['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
close_prices_scaled = scaler.fit_transform(close_prices)

In [4]:
def create_lstm_data(data, time_steps=1):
    x, y = [], []
    for i in range(len(data) - time_steps):
        x.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps, 0])
    return np.array(x), np.array(y)

In [5]:
time_steps = 10
x, y = create_lstm_data(close_prices_scaled, time_steps)
x = np.reshape(x, (x.shape[0], x.shape[1], 1))

In [6]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [7]:
model.fit(x, y, epochs=50, batch_size=32)

Epoch 1/50

8/8 [==============================] - 3s 7ms/step - loss: 0.0709
Epoch 2/50
8/8 [==============================] - 0s 6ms/step - loss: 0.0164
Epoch 3/50
8/8 [==============================] - 0s 6ms/step - loss: 0.0125
Epoch 4/50
8/8 [==============================] - 0s 6ms/step - loss: 0.0096
Epoch 5/50
8/8 [==============================] - 0s 5ms/step - loss: 0.0057
Epoch 6/50
8/8 [==============================] - 0s 6ms/step - loss: 0.0049
Epoch 7/50
8/8 [==============================] - 0s 5ms/step - loss: 0.0044
Epoch 8/50
8/8 [==============================] - 0s 5ms/step - loss: 0.0045
Epoch 9/50
8/8 [==============================] - 0s 5ms/step - loss: 0.0045
Epoch 10/50
8/8 [==============================] - 0s 6ms/step - loss: 0.0042
Epoch 11/50
8/8 [==============================] - 0s 5ms/step - loss: 0.0042
Epoch 12/50
8/8 [==============================] - 0s 6ms/step - loss: 0.0042
Epoch 13/50
8/8 [==============================] - 0s 6ms/step - loss: 0

In [8]:
future_dates = pd.date_range(start=end_date, periods=30)
last_prices = close_prices[-time_steps:]
last_prices_scaled = scaler.transform(last_prices.reshape(-1, 1))
x_pred = np.array([last_prices_scaled[-time_steps:, 0]])
x_pred = np.reshape(x_pred, (x_pred.shape[0], x_pred.shape[1], 1))
predicted_prices_scaled = model.predict(x_pred)
predicted_prices = scaler.inverse_transform(predicted_prices_scaled)

1/1 [==============================] - 1s 848ms/step


In [9]:
future_data = pd.DataFrame({'Date': future_dates, 'Predicted Price': predicted_prices.flatten()})
print(future_data)

ValueError: All arrays must be of the same length